In [1]:
import autosklearn.classification
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import sklearn.metrics

In [2]:
train = pd.read_csv('../Datasets/SPECT Heart/spect_train_binary.csv')
test = pd.read_csv('../Datasets/SPECT Heart/spect_test_binary.csv')

In [3]:
X_train = train.drop('OVERALL_DIAGNOSIS', axis=1)
y_train = train['OVERALL_DIAGNOSIS']
X_test = test.drop('OVERALL_DIAGNOSIS', axis=1)
y_test = test['OVERALL_DIAGNOSIS']


# AUTOML

In [9]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

In [10]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.8128342245989305

In [11]:
import pickle
with open('./models/spect_heart_automl.pkl', 'wb') as f:
    pickle.dump(automl, f)

In [1]:
import pickle
with open('./models/spect_heart_automl.pkl', 'rb') as f:
    automl = pickle.load(f)

automl.show_models()

{300: {'model_id': 300,
  'rank': 1,
  'cost': 0.07407407407407407,
  'ensemble_weight': 0.02,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7f476c1f3b50>,
  'balancing': Balancing(random_state=1, strategy='weighting'),
  'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice at 0x7f476c143a50>,
  'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice at 0x7f476c14f1d0>,
  'sklearn_classifier': ExtraTreesClassifier(criterion='entropy', max_features=3, min_samples_leaf=10,
                       min_samples_split=4, n_estimators=512, n_jobs=1,
                       random_state=1, warm_start=True)},
 301: {'model_id': 301,
  'rank': 2,
  'cost': 0.07407407407407407,
  'ensemble_weight': 0.06,
  'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice at 0x7f476b97ff50>,
  'balancing': Balancing(random_state=1, strategy

# RANDOM FOREST

In [12]:
import sklearn.ensemble

model = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5, random_state=42)
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

In [13]:
sklearn.metrics.accuracy_score(y_test, y_hat)

0.7647058823529411

# SVC

In [13]:
import sklearn.svm
import sklearn.model_selection

model = sklearn.svm.SVC(random_state=42)
params = {'C': [0.1, 1, 10, 100, 1000], 'gamma': [1, 0.1, 0.01, 0.001, 0.0001], 'kernel': ['linear', 'ploy', 'rbf', 'sigmoid'], 'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}
gridSearch = sklearn.model_selection.GridSearchCV(model, param_grid=params, cv=5, n_jobs=5, verbose=3)
gridSearch.fit(X_train, y_train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/model_selection/_search.py:925: UserWarning: One or more of the test scores are non-finite: [0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    nan 0.6625 0.6625
 0.7375    nan 0.625  0.7375 0.7375    nan 0.7    0.6875 0.7375    nan
 0.6625 0.6625 0.7375    nan 0.6625 0.6625 0.7375    n

GridSearchCV(cv=5, estimator=SVC(random_state=42), n_jobs=5,
             param_grid={'C': [0.1, 1, 10, 100, 1000],
                         'degree': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                         'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                         'kernel': ['linear', 'ploy', 'rbf', 'sigmoid']},
             verbose=3)

In [14]:
y_hat = gridSearch.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

0.8449197860962567

In [11]:
import pickle
with open('./models/spect_heart_gridsearch.pkl', 'wb') as f:
    pickle.dump(gridSearch, f)

# XGBOOST

In [20]:
import xgboost

xgb = xgboost.cv({'n_estimators': 1000, 'max_depth': 6, 'eta': 0.1, 'silent': 1, 'objective': 'binary:logistic', 'eval_metric': 'error'}, xgboost.DMatrix(X_train, y_train), num_boost_round=100, early_stopping_rounds=10, nfold=5)

[14:56:09] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:56:09] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[14:56:09] WARNING: ../src/learner.cc:627: 
Parameters: { "n_estimators", "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or som

In [21]:
xgb

,train-error-mean,train-error-std,test-error-mean,test-error-std
0,0.187500,0.026146,0.4000,0.115920
1,0.184375,0.026882,0.3375,0.030619
2,0.187500,0.032775,0.3375,0.050000
3,0.178125,0.028980,0.3000,0.100000
4,0.168750,0.030298,0.2875,0.101550
5,0.175000,0.033366,0.2750,0.101550
6,0.171875,0.035630,0.2750,0.101550
7,0.168750,0.030298,0.2625,0.082916
8,0.171875,0.035630,0.2750,0.101550
9,0.159375,0.025000,0.2375,0.091856


In [23]:
model = xgboost.XGBClassifier(n_estimators=1000, max_depth=6, eta=0.1, silent=1, objective='binary:logistic', eval_metric='error')

In [24]:
model.fit(X_train, y_train)
y_hat = model.predict(X_test)
sklearn.metrics.accuracy_score(y_test, y_hat)

[14:57:16] WARNING: ../src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




0.679144385026738

# LOAD MODEL

In [5]:
import pickle
with open('./models/spect_heart_gridsearch.pkl', 'rb') as f:
    model = pickle.load(f)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator GridSearchCV from version 1.2.2 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# CIU

In [6]:
from ciu import determine_ciu
import tqdm
import metrics

In [7]:
def enc_exp(exp, feature_num):
    enc_exp = np.zeros((len(exp),feature_num))
    for i in range(len(exp)):
        for j in range(len(exp[i])):
            enc_exp[i][int(exp[i,j,0])] = exp[i,j,1]
    return enc_exp

In [8]:
feat_list = X_train.columns.tolist()

In [9]:
def exp_fn_blk(xtest):
    exp1 = []
    for i in tqdm.tqdm(range(len(xtest))):
        exp = determine_ciu(X_test.iloc[i:i+1], model.predict_proba, X_train.to_dict('list'), samples = 1000, prediction_index = 1)
        exp_list = [[feat_list.index(i), exp.ci[i]] for i in exp.ci]
        exp1.append(exp_list)
    return np.array(exp1)

In [10]:
exp1 = exp_fn_blk(X_test)
exp2 = exp_fn_blk(X_test)

100%|██████████| 187/187 [00:46<00:00,  4.05it/s]


In [12]:
np.save('./explanations/spect_hear_ciu1.npy', exp1)
np.save('./explanations/spect_hear_ciu2.npy', exp2)

In [13]:
i = metrics.calc_identity(exp1, exp2)
s = metrics.calc_separability(exp1)
enc1 = enc_exp(exp1, len(feat_list))
sb = metrics.calc_stability(enc1, y_test)

/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [14]:
i, s, sb

((100.0, 0, 187), (0, 187, 34969, 0.0), (21, 187))

In [15]:
X_test_norm = metrics.normalize_test(X_train, X_test)
sim = metrics.calc_similarity(exp1, X_test_norm)

In [16]:
sim

0.0037571007712553744

In [17]:
list_monotonicity = []
list_non_sensitivity = []
list_effective_complexity = []

for i in tqdm.tqdm(range(len(X_test))):
    atr = exp1[i]
    sorted_atr = [j for i,j in atr]
    sorted_feat = [i for i,j in atr]
    y = np.zeros(2, dtype=int)
    np.put(y, y_test.iloc[i], 1)
    example = metrics.FeatureAttribution(model, X_test.to_numpy()[i], y, sorted_atr)
    list_monotonicity.append(example.monotonicity())
    list_non_sensitivity.append(example.non_sensitivity())
    list_effective_complexity.append(example.effective_complexity(sorted_feat, 0.1))

100%|██████████| 187/187 [00:03<00:00, 53.18it/s]


In [18]:
print(np.mean(list_monotonicity))
print(np.mean(list_non_sensitivity))
print(np.mean(list_effective_complexity))

print(np.median(list_monotonicity))
print(np.median(list_non_sensitivity))
print(np.median(list_effective_complexity))

0.22138943677544293
0.0
0.0
0.15979672501411638
0.0
0.0


In [19]:
metrics.calc_trust_score(model, X_test.to_numpy(), exp1, 3, X_train.columns.to_list())

100%|██████████| 187/187 [00:01<00:00, 122.94it/s]


0.2691622103386809

# RULEFIT

In [4]:
from skrules import SkopeRules
import metrics_rules
import time

In [5]:
clf = SkopeRules(max_depth_duplication=2,
                    n_estimators=512,
                    precision_min=0.3,
                    recall_min=0.1,
                    feature_names=X_train.columns.tolist())

In [6]:
start_time = time.time()
clf.fit(X_train, y_train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 12.452015161514282 seconds ---


In [7]:
start_time = time.time()
top_rules1 = clf.score_top_rules(X_test)
top_rules2 = clf.score_top_rules(X_test)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.08147931098937988 seconds ---


In [8]:
i = metrics_rules.calc_identity_rules(top_rules1, top_rules2)
print(i)

s = metrics_rules.calc_separability_rules(top_rules1)
print(s)

enc_rules = metrics_rules.exp_enc(clf, top_rules1)
sb = metrics_rules.calc_stability_rules(enc_rules, y_test)
print(sb)

(0.0, 187, 187)
(11358, 187, 34969, 32.48019674568904)
(68, 187)


/home/krish/anaconda3/envs/interpret/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [9]:
X_test_norm = metrics_rules.normalize_test(X_train, X_test)
sim = metrics_rules.calc_similarity(enc_rules, X_test_norm)
print(sim)

0.0
